## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2025, day=9)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import time_solution

## Part a
Looks like pair-wise distance calculations are needed again. I'll start with the scipy pdist implementation this time.

In [7]:
# Imports
import numpy as np
from scipy.spatial.distance import pdist

In [24]:
# Functions
def parse_input(input_data: str) -> np.ndarray:
    """Parse input data into a numpy array of box coordinates."""
    return np.loadtxt(input_data.splitlines(), delimiter=",", dtype=np.int32)


def find_furthest_pair_indices(pts: np.ndarray, k: int = 1_000) -> list[tuple[int, int]]:
    """Find the k furthest pairs for an array of points."""
    # Calculate pairwise distances using scipy pdist
    dists = pdist(pts, metric="euclidean")

    # k cannot exceed number of distances
    k = min(k, len(dists))

    # Reverse the ordering
    partitioned = np.argpartition(-dists, k - 1)[:k]
    sorted_k = partitioned[np.argsort(-dists[partitioned])]

    # Get upper triangle indices
    i_idx, j_idx = np.triu_indices(len(pts), k=1)

    # Return the indices of the closest pairs in the original point-array
    return list(zip(i_idx[sorted_k], j_idx[sorted_k], strict=True))


def calculate_furthest_box_area(input_data: str) -> int:
    """Calculate the area of the rectangle defined by the furthest box pair."""
    # Load input data
    tiles = parse_input(input_data)

    # Get the furthest box pairs using a common scipy-based function
    furthest_box_pairs = find_furthest_pair_indices(tiles)

    # Get the inclusive rectangle dimensions defined by the furthest tile pair
    a, b = furthest_box_pairs[0]
    diff = np.abs(tiles[a] - tiles[b]) + np.array([1, 1])

    # Calculate the area of the rectangle defined by the furthest tile pair
    return int(np.prod(diff.astype(np.int32)))

Again something went wrong with fetching the example answer. I've pasted the correct answer below.

In [25]:
# Correctness check
calculate_furthest_box_area(example.input_data) == 50

True

In [ ]:
# Performance check
scipy_time_a = time_solution(calculate_furthest_box_area, input_data)
print(f"The scipy implementation takes {scipy_time_a:.1f} ms per run.")

The scipy implementation takes 1.1 ms per run.


In [68]:
# Submit answer
puzzle.answer_a = calculate_furthest_box_area(input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


## Part b
I use shapely polygon objects to represent the main polygon described by the red tiles and calculate the areas of the rectangles within it.

This is however not very performant, at roughly 3 seconds for my input data. I'll see if I can optimize it later.

In [5]:
from shapely.geometry import Polygon


def largest_area_within_red_tiles(input_data: str) -> int:
    """Find the largest area rectangle that can fit inside the polygon defined by the red tiles."""
    red_tiles = parse_input(input_data)

    # Construct the main polygon that is circumscribed by the red tiles
    main_polygon = Polygon(red_tiles)

    # Initialize the largest area found
    largest_area = 0

    # Iterate over all pairs of red tiles to define rectangles
    for i, (xa, ya) in enumerate(red_tiles):
        for _, (xb, yb) in enumerate(red_tiles[i + 1 :]):
            # Ensure we have a valid rectangle
            if xa != xb and ya != yb:
                # Construct the rectangle as a polygon
                rectangle = Polygon([(xa, ya), (xa, yb), (xb, yb), (xb, ya)])
                if main_polygon.contains(rectangle):
                    # Calculate area, inclusive of edges
                    rectangle_are = (abs(xa - xb) + 1) * (abs(ya - yb) + 1)
                    largest_area = max(rectangle_are, largest_area)

    return int(largest_area)

In [17]:
# Correctness check
largest_area_within_red_tiles(example.input_data) == 24

True

In [ ]:
# Performance check
shapely_time_b = time_solution(largest_area_within_red_tiles, input_data, iterations=10, runs=3, time_unit="s")
print(f"The shapely implementation takes {shapely_time_b:.1f} s per run.")

The shapely implementation takes 3.1 s per run.


In [29]:
# Submit answer
puzzle.answer_b = largest_area_within_red_tiles(input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
